In [1]:
from selenium import webdriver
import re
import pandas as pd
import csv
import time

In [2]:
Chrome = webdriver.Chrome
chromedriver = './chromedriver'
browser = Chrome(chromedriver)

In [3]:
# Table headers for the csv file
table_headers = ['Song', 'SongTitleEnglish', 'SongTitleHindi', 'Year','MovieEnglish', 'MovieHindi',\
                 'Music Director', 'Lyricist', 'SingersEnglish','SingersHindi']

In [4]:
# Opening the file in write mode and hence creating a new file with just the headers
with open(r'hindilyrics_pratik.csv', 'w') as file:
    writer = csv.writer(file)
    writer.writerow(table_headers)

In [5]:
url = 'https://hindilyricspratik.blogspot.com/search?max-results=200'

In [ ]:
%%time
page_counter = 0
song_counter = 0
while True:
    try:
        print('Opening new page')
        browser.get(url)
        page_counter += 1
    except Exception as e:
        print(e)
        # Extracting information about all posts on the page
        posts = browser.find_elements_by_css_selector('div.post-outer')

        for i in range(len(posts)):
            try:
                # Getting post title and post body for each post
                post_title = posts[i].find_element_by_css_selector('h3.post-title').text
                post_body = posts[i].find_element_by_css_selector('div.post-body').text


                # Getting various data
                title_hindi = post_title.split(' - ')[0]
                title_english = post_title.split(' - ')[1].split(' (')[0]
                singers_english = ', '.join(re.findall("\((.*?)\)", post_title)[0].split(',')[:-1])
                movie_english = re.findall("\((.*?)\)", post_title)[0].split(',')[-1]

                lyrics = '\n\n'.join(post_body.split('\n\n')[1:])

                post_meta = post_body.split('\n')[:5]

                movie_hindi = post_meta[0].split(': ')[1].split(' (')[0]
                year = int(re.findall("\((.*?)\)", post_meta[0].split(': ')[1])[0])
                music_director = post_meta[1].split(': ')[1]
                lyricist = post_meta[2].split(': ')[1]
                singers_hindi = post_meta[3].split(': ')[1]

                song_counter += 1

                print(f'Printing song {song_counter} on page {page_counter}: {title_english}')

                song_data = [lyrics, title_english, title_hindi, year, movie_english, movie_hindi, \
                             music_director, lyricist, singers_english, singers_hindi]

                # Writing the data for each song to the csv file
                # Notice that this time the file was opened in append mode
                with open(r'hindilyrics_pratik.csv', 'a') as file:
                    writer = csv.writer(file)
                    writer.writerow(song_data)
            except Exception as e:
                print(e)

        try:
            url = browser.find_element_by_id('Blog1_blog-pager-older-link').get_attribute('href')
            page_counter += 1
        except Exception as e:
            print('No older page', e)
            break
    finally:
        time.sleep(10)

Opening new page
